### Project Euler Problem: 102

Three distinct points are plotted at random on a Cartesian plane, for which -1000 ≤ x, y ≤ 1000, such that a triangle is formed.

Consider the following two triangles:

A(-340,495), B(-153,-910), C(835,-947)

X(-175,41), Y(-421,-714), Z(574,-645)

It can be verified that triangle ABC contains the origin, whereas triangle XYZ does not.

Using triangles.txt (right click and 'Save Link/Target As...'), a 27K text file containing the co-ordinates of one thousand "random" triangles, find the number of triangles for which the interior contains the origin.

NOTE: The first two examples in the file represent the triangles in the example given above.

### Solution: ###

The strategy here is to begin by noticing that any triangle, which contains $\mathbb{0}$<br>
in its interior, extends to all four quadrants of its graph.<p>

To implement these ideas, I begin by importing **triangles.txt** as a CSV.


In [41]:
import qualified Data.ByteString.Lazy as BL
import Data.Vector (Vector, empty, toList)
import Data.Either.Extra (fromRight)
import Data.Csv

csv <- BL.readFile "triangles.txt"
parsedCsv = decode NoHeader csv :: Either String (Vector [Integer])

triangles = map f $ toList.fromRight empty $ parsedCsv
    where
        f [a,b,c,d,e,f] = [(a,b),(c,d),(e,f)]

In [42]:
head triangles

[(-340,495),(-153,-910),(835,-947)]

Next, assign their type a type synonym,<br>
create a Quadrants datatype, and give this datatype some useful methods.<br>

In [37]:
type Triangle = [(Integer, Integer)]
data Quadrant = I | II | III | IV deriving (Eq, Show)

toQuad :: Triangle -> [Quadrant]
toQuad [] = []
toQuad (pt:pts) = f pt : toQuad pts
  where
    f (x,y) | and [x >= 0, y >= 0] = I
    f (x,y) | and [x <= 0, y >= 0] = II
    f (x,y) | and [x <= 0, y <= 0] = III
    f (x,y) | otherwise = IV

coords :: Triangle -> [Integer]
coords [(a,b),(c,d),(e,f)] = [a,b,c,d,e,f]

In [43]:
coords.head $ triangles
toQuad.head $ triangles


[-340,495,-153,-910,835,-947]

[II,III,IV]